
#  **Descarga y procesamiento de imagenes DEM y LULC, y generacion de mapas de distancias a objetos de interes, a utilizar como isnumos para el modelo de inundaciones** 

Este notebook presenta:

1- codigo para descarga de imagenes DEM-SRTM desde la plataforma Google Earth Engine (GEE) y su posterior procesamiento para la conformacion del dataset a utilizar en el modelo de susceptibilidad a inundaciones. Es importante aclarar que para poder correr este notebook se debe estar registrado en GEE; si es necesario registrarse ingresar en https://earthengine.google.com/ y seguir los pasos alli detallados;

2- procesamiento del mapa de Cobertura y Uso de Suelo 2021 (LULC 2021) elaborado por ESRI y descargados desde https://www.arcgis.com/apps/instant/media/index.html?appid=fc92d38533d440078f17678ebc20e8e2;

3- generacion de mapas de distancia a objetos de interes (rutas, rios), los cuales fueron generados en formato vectorial a partir del pligin de OpenStreetMaps de QGIS;

4- compilacion de factores condicionantes en una unica imagen con 6 bandas.



# **1. Descarga de DEM**


##  **1.1. Instalación e importación de librerias**

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [ ]:
!apt install gdal-bin python-gdal python3-gdal 

In [ ]:
import ee
import os
from geemap import geojson_to_ee, ee_to_geojson
from osgeo import gdal
import numpy as np
import shutil

## **1.2. Autenticacion e Inicio de Google Earth Engine**

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## **1.3. Declaración de variables**

In [ ]:
#bucket con archivo vectorial del area de interes para modelo de inundaciones
BUCKET_AOI = "gs://dym-workshops-public/immap/inundaciones/aux_data/aoi/*.geojson" 

#creamos las carpeta a donde descargaremos los archivos desde bucket
BASE_PATH = "drive/MyDrive/IMMAP/flood_susceptibility/data/"
PATH_AOI = f"{BASE_PATH}/aoi_flood/"
DEM_PATH = f"{BASE_PATH}/DEM/"

## **1.4. Descarga de archivos desde bucket**

In [ ]:
!mkdir -p $PATH_AOI   #crea la carpeta
!mkdir -p $DEM_PATH

!gsutil -m cp -r $BUCKET_AOI   $PATH_AOI # descarga de imagenes

## **1.5. Descarga de imagenes DEM**

In [ ]:
for file in os.listdir(PATH_AOI):
    aoi = (geojson_to_ee(f'{PATH_AOI}{file}')).geometry()

    srtm = ee.Image('USGS/SRTMGL1_003').clip(aoi)

    # Oprcion 1: descarga de imagen S1 por departamento a Google Drive
    task = ee.batch.Export.image.toDrive(
        image= srtm,  
        region= aoi,  
        description = 'srtm_export',
        folder = 'DEM',
        fileNamePrefix= f'dem_srtm_espg32618',
        #maxPixels: 1e15, 
        scale=5000,  #cambiar escala a 500 m
        crs='EPSG:32618')
    
    # Oprcion 2: descarga de imagen a bucket
    #task = ee.batch.Export.image.toCloudStorage(
    #    image = srtm,
    #    bucket = '{BUCKET_DEM}',  #completar con nombre BUCKET
    #    fileNamePrefix = f"{FILE_NAME}",  #completar con file name
    #    maxPixels = 8030040147504,
    #    scale = 500,
    #    region = aoi
    #    crs='EPSG:32618')                              
    
    task.start()

    import time 
    while task.active():
        print('Polling for task (id: {}).'.format(task.id))
        time.sleep(5)

In [ ]:
#verificar que la imagen DEM fue correctamente descargada
!ls $DEM_PATH

## **1.6. Preparacion del DEM**

Se deben setear los valores no dato como nan

In [ ]:
for file in os.listdir(DEM_PATH):
    filename = f'{DEM_PATH}{file}'
    dem = gdal.Open(filename)
    cols = dem.RasterXSize 
    rows = dem.RasterYSize 
    proj = dem.GetProjection()
    geotr = dem.GetGeoTransform();
    dem_raster = dem.GetRasterBand(1)
    noDataVal = dem_raster.GetNoDataValue()
    
    dem_array = dem.GetRasterBand(1).ReadAsArray(0,0,cols,rows).astype(float) 
    dem_nodata = dem_array
    dem_nodata[np.where(dem_array == noDataVal)] = np.nan
    
    #save new file
    driver = gdal.GetDriverByName('GTiff')
    file = driver.Create(f'{DEM_PATH}dem_srtm_500m_32618_final.tif', cols, rows, 1, gdal.GDT_Float32)
    file.GetRasterBand(1).WriteArray(dem_nodata)
    file.GetRasterBand(1).SetNoDataValue(np.nan)

    # spatial ref system
    file.SetProjection(proj)
    file.SetGeoTransform(geotr)
    file.FlushCache()

In [ ]:
!ls $DEM_PATH

## **1.7. Generacion de mapas derivados del DEM**

A continuacion se describen los pasos para generar los mapas de pendiente y el Indice Topografico de Humedad (TWI, por sus siglas en ingles), a utilizar como inputs del modelo de suceptibilidad a inundaciones.

*Importante:* el mapa de elevacion debe estar proyectado.

### <font color='green'> **TWI = parametro utilizado para identificar areas donde se tiende a acumular el agua teniendo en cuenta como fluye el agua de acuerdo con la pendiente que haya en los pixeles que rodean al pixel de interes. Se calcuala como: TWI = ln (As/tanß), donde As representa 'Specific Catchment Area'(parametro usualmente utilizado en hidrologia para analizar el flujo de agua en las laderas de montañas) y beta representa la pendiente. </font>

In [ ]:
#PARA CORRER LAS SIGUIENTES LINEAS ES NECESARIO TENER INSTALADA LA LIBRERIA SAGA
!sudo apt update
!sudo apt install saga

In [ ]:
import glob
import subprocess

## **1.7.a. Estimacion de mapa de pendiente (°)**

https://saga-gis.sourceforge.io/saga_tool_doc/2.2.3/ta_morphometry_0.html

In [ ]:
dem_file = f'{DEM_PATH}dem_srtm_500m_32618_final.tif'
slope_file = f'{DEM_PATH}slope_degrees.sdat'
aspect_file  = f'{DEM_PATH}aspect_degrees.sdat'
cmd_morphometry = f'saga_cmd ta_morphometry 0 -ELEVATION {dem_file} -SLOPE {slope_file} -ASPECT {aspect_file} -METHOD {6} -UNIT_SLOPE {1} -UNIT_ASPECT {1}'
subprocess.run(cmd_morphometry, shell=True)

In [ ]:
#paso el archivo a GeoTiff
#Module Export Raster https://saga-gis.sourceforge.io/saga_tool_doc/2.3.0/io_gdal_1.html
slope_file_tif = f'{DEM_PATH}slope_degrees.tif'
cmd_slope_export = f'saga_cmd io_gdal 1 -GRIDS {slope_file} -FILE {slope_file_tif} -FORMAT {1} -TYPE {6} -SET_NODATA {1} -NODATA {np.nan}' 
subprocess.run(cmd_slope_export, shell=True)

## **1.7.b. Estimacion de mapa de pendiente (rad) para posterior estimacion de TWI**

In [ ]:
#leo archivo de pendiente
infile = f'{DEM_PATH}slope_degrees.tif'
raster = gdal.Open(infile)
noDataVal = raster.GetRasterBand(1).GetNoDataValue() 
array = raster.GetRasterBand(1).ReadAsArray(0,0,raster.RasterXSize,raster.RasterYSize).astype(float) 
reclass = array
reclass[np.where(array == 0)] = 0.01 # change 0 values to 0.01
reclass = reclass*0.01745  #transform to radian values

#save slope in radian file
driver = gdal.GetDriverByName('GTiff')
file = driver.Create(f'{DEM_PATH}slope_radians.tif', raster.RasterXSize, raster.RasterYSize , 1, gdal.GDT_Float32)
file.GetRasterBand(1).WriteArray(reclass)
file.GetRasterBand(1).SetNoDataValue(np.nan)

# spatial ref system
file.SetProjection(raster.GetProjection())
file.SetGeoTransform(raster.GetGeoTransform())
file.FlushCache()

## **1.7.b. Estimacion de TWI**

Para estimar TWI, los pasos a seguir son:

1-Fill Sinks: remover depresiones locales (sinks) - necesario para estimar Flow Accumulation (o direccion de flujo) 
https://saga-gis.sourceforge.io/saga_tool_doc/2.2.3/ta_preprocessor_4.html

2-Flow Accumulation (Top-Down) (aka Flow Catchment area // Acumulacion de Flujo) - se utiliza para identificar planicies o cuencas hidrograficas. Por ejemplo, para cada celda, la acumulacion de flujo esta determinada por cuantas celdas fluyen a traves de esta celda; si el valor de acumulacion de flujo es mayor, el area sera mas facil para formar una escorrentia. Se utiliza para estimar Specific Catchment area.
https://saga-gis.sourceforge.io/saga_tool_doc/2.3.0/ta_hydrology_0.html

3-Specific Catchment area: parametro usualmente utilizado en hidrologia para analizar el flujo de agua en las laderas de monta;as. Insumo de TWI. 
https://saga-gis.sourceforge.io/saga_tool_doc/2.2.3/ta_hydrology_19.html

4-TWI: insumos - Specific Catchment Area + Slope (radians).
https://saga-gis.sourceforge.io/saga_tool_doc/2.2.3/ta_hydrology_20.html

In [ ]:
#Fill sinks
filled_file = f'{DEM_PATH}DEM_aoi_filled_sink.sdat'
cmd_sinks = f"saga_cmd ta_preprocessor 5 -ELEV {dem_file} -FILLED  {filled_file} -MINSLOPE {0.01}"
subprocess.run(cmd_sinks, shell=True)

In [ ]:
# Flow accumulation
fl_accu = f'{DEM_PATH}flow_accu_mt4_fillsink0pt001.sdat'
mean_over_catchment = f'{DEM_PATH}mean_over_catchment.sdat'
cmd_flow_accumulation = f'saga_cmd ta_hydrology 0 -ELEVATION {filled_file} -FLOW {fl_accu} -VAL_MEAN {mean_over_catchment} -ACCU_TARGET {fl_accu} -STEP {1} -FLOW_UNIT {1} -METHOD {4} -LINEAR_DO {0} -LINEAR_MIN {500} -CONVERGENCE {1.1000000} -NO_NEGATIVES {1}'
subprocess.run(cmd_flow_accumulation, shell=True)

In [ ]:
#Specific Catchment Area
flow_width = f'{DEM_PATH}fl_width.sdat'
sp_cath_area = f'{DEM_PATH}sca.sdat'
cmd_sca = f'saga_cmd ta_hydrology 19 -DEM {filled_file} -WIDTH {flow_width} -TCA {fl_accu} -SCA {sp_cath_area} -METHOD {2}'
subprocess.run(cmd_sca, shell=True)

In [ ]:
#TWI
twi = f'{DEM_PATH}twi_topmodel.sdat'
slope_rad_file = f'{DEM_PATH}slope_radians.tif'
cmd_twi = f'saga_cmd ta_hydrology 20 -SLOPE {slope_rad_file} -AREA {sp_cath_area} -TWI {twi} -CONV {0} -METHOD {1}'
subprocess.run(cmd_twi, shell=True)

In [ ]:
#convierto a .tif
twi_file_tif = f'{DEM_PATH}twi_topmodel.tif'
cmd_twi_export = f'saga_cmd io_gdal 1 -GRIDS {twi} -FILE {twi_file_tif} -FORMAT {1} -TYPE {6} -SET_NODATA {1} -NODATA {np.nan}'
subprocess.run(cmd_twi_export, shell=True)

In [ ]:
#delete saga temporary files
args_sdat = ('rm', '-rf', f'{DEM_PATH}*.sdat')
args_mgrd = ('rm', '-rf', f'{DEM_PATH}*.mgrd')
args_prj = ('rm', '-rf', f'{DEM_PATH}*.prj')
args_xml = ('rm', '-rf', f'{DEM_PATH}*.xml')
args_sgrd = ('rm', '-rf', f'{DEM_PATH}*.sgrd')
subprocess.call('%s %s %s' % args_sdat, shell=True)
subprocess.call('%s %s %s' % args_mgrd, shell=True)
subprocess.call('%s %s %s' % args_prj, shell=True)
subprocess.call('%s %s %s' % args_xml, shell=True)
subprocess.call('%s %s %s' % args_sgrd, shell=True)

-----------------------------------------

# **2. Procesamiento LULC para modelo inundaciones**

##  **2.1. Instalación e importación de librerias**

In [ ]:
!pip install geopandas

In [ ]:
from pathlib import Path
import glob
from os import remove
import geopandas as gpd

## **2.2. Declaración de variables**

In [ ]:
#bucket con los archivos a utilizar
#LULC
BUCKET_LULC = "gs://dym-workshops-public/immap/asentamientos/aux_data/LULC/*.tif"

#creamos las carpetas a donde descargaremos los archivos desde el bucket
BASE_PATH = "drive/MyDrive/IMMAP/flood_susceptibility/data/"
PATH_LULC_FILES = f"{BASE_PATH}LULC_files/"

#carpeta de guardado de las mascara de area construida
PATH_LULC_MASK = f"{BASE_PATH}/LULC_MASK/"

#PARAMETROS GENERALES
aoi = 'colombia'
prj = "32618"

## **2.3. Descarga de archivos desde bucket**

In [ ]:
#LULC
!mkdir -p $PATH_LULC_FILES   #crea la carpeta
!gsutil -m cp -r $BUCKET_LULC   $PATH_LULC_FILES # descarga de archivos'''

## **2.4. Generar LULC 500m**

In [ ]:
for file in os.listdir(PATH_LULC_FILES):
    print(file)

In [ ]:
#resample to 500m
for file in os.listdir(PATH_LULC_FILES):
    if file.endswith('.tif'):
        infile_resample = f'{PATH_LULC_FILES}{file}'
        outfile_resample = f'{PATH_LULC_FILES}{file[0:-4]}_500m.vrt'
        ds_subset = gdal.Open(infile_resample)
        dsResampl = gdal.Warp(outfile_resample, ds_subset, xRes = 5000, yRes = 5000) #CAMBIAR xRes/yRes a 500
        dsResampl = None 

In [ ]:
# unificar la proyeccion de los archivos LULC
for file in os.listdir(PATH_LULC_FILES):
    if file.endswith('_500m.vrt'):
        ds = gdal.Open(f'{PATH_LULC_FILES}{file}')
        output_file = f'{PATH_LULC_FILES}{file[0:-4]}_epsg{prj}.vrt'
        dsReprj = gdal.Warp(output_file, ds, dstSRS=f'EPSG:{prj}', format='VRT')
        #ds = None
        #dsReprj = None

In [ ]:
outfile_mosaic = f'{PATH_LULC_FILES}lulc2021_mosaic_500m.tif'
files_to_mosaic = glob.glob(os.path.join(Path(PATH_LULC_FILES, "*_epsg32618.vrt")))
m = gdal.Warp(outfile_mosaic, files_to_mosaic, format = "GTiff", options =["TILED=YES"])
m = None 
del m

In [ ]:
#subset by aoi
for file in os.listdir(PATH_AOI):
    gdf = f'{PATH_AOI}{file}'
    infile_subset = f'{PATH_LULC_FILES}lulc2021_mosaic_500m.tif'
    outfile_subset = f'{PATH_LULC_FILES}lulc2021_500m_aoi.tif'
    dataset_subset = gdal.Open(infile_subset)

    ds_recorte = gdal.Warp(outfile_subset, dataset_subset , cutlineDSName = gdf, cropToCutline = True, dstNodata = np.nan)
    ds_recorte = None

In [ ]:
# set no data as nan and mask out non urban classes
raster = gdal.Open(f'{PATH_LULC_FILES}lulc2021_500m_aoi.tif')
noDataVal = raster.GetRasterBand(1).GetNoDataValue() 
array = raster.GetRasterBand(1).ReadAsArray(0,0,raster.RasterXSize,raster.RasterYSize).astype(float) 
reclass = array
reclass[np.where(array == noDataVal)] = np.nan
    
#save new file
driver = gdal.GetDriverByName('GTiff')
file = driver.Create(f'{PATH_LULC_FILES}lulc2021_500m_final.tif', raster.RasterXSize,raster.RasterYSize, 1, gdal.GDT_Float32)
file.GetRasterBand(1).WriteArray(reclass)
file.GetRasterBand(1).SetNoDataValue(np.nan)

# spatial ref system
file.SetProjection(raster.GetProjection())
file.SetGeoTransform(raster.GetGeoTransform())
file.FlushCache()

In [ ]:
for f in glob.glob(os.path.join(Path(PATH_LULC_FILES, "*.vrt"))):
    os.remove(f)

for f in glob.glob(os.path.join(Path(PATH_LULC_FILES, "*.tif"))):
    if f.endswith('_aoi.tif'):
        os.remove(f)
    else:
        pass

-------------------------------

# **3. Generacion de mapas de distancias a objetos de interes**

##  **3.1. Instalación e importación de librerias**

In [ ]:
!pip install rasterio

In [ ]:
import osr
import sys
import rasterio

## **3.2. Declaración de variables**

In [ ]:
#bucket con archivo vectorial del area de interes para modelo de inundaciones
BUCKET_OSM = "gs://dym-workshops-public/immap/inundaciones/aux_data/OSM/*.gpkg" 

#creamos las carpeta a donde descargaremos los archivos desde bucket
BASE_PATH = "drive/MyDrive/IMMAP/flood_susceptibility/data/"
PATH_OSM = f"{BASE_PATH}/OSM/"

#PARAMETROS GENERALES
prj = "32618"

## **3.3. Descarga de archivos desde bucket**

In [ ]:
!mkdir -p $PATH_OSM   #crea la carpeta

!gsutil -m cp -r $BUCKET_OSM   $PATH_OSM # descarga de imagenes

In [ ]:
!ls $PATH_OSM

## **3.4. Generacion de rasters vacios**

Se generan rasters vacios sobre los que despues se van a calcular las distancias a los objetos de interes.

In [ ]:
#se lee un raster de base para obtener parametros generales a utilizar en el raster vacio
base_raster = gdal.Open(f'{DEM_PATH}dem_srtm_500m_32618_final.tif') 
col = base_raster.RasterXSize 
row = base_raster.RasterYSize 
gtr = base_raster.GetGeoTransform()
nodata = np.nan

spatref = osr.SpatialReference()
spatref.ImportFromEPSG(int(prj))
wkt = spatref.ExportToWkt()

#raster vacio para calcular distancia a rios
output_empty_file_rivers = f'{PATH_OSM}distance_from_river.tif'

driver = gdal.GetDriverByName('GTiff')
empty_file = driver.Create(output_empty_file_rivers, col, row, 1, gdal.GDT_Float32)
empty_file.SetProjection(wkt)
empty_file.SetGeoTransform(gtr)
empty_file.GetRasterBand(1).Fill(0)
empty_file.GetRasterBand(1).SetNoDataValue(nodata)
empty_file.FlushCache()
empty_file = None

#raster vacio para calcular distancia a calles
output_empty_file_roads = f'{PATH_OSM}distance_from_roads.tif'

empty_file_roads = driver.Create(output_empty_file_roads, col, row, 1, gdal.GDT_Float32)
empty_file_roads.SetProjection(wkt)
empty_file_roads.SetGeoTransform(gtr)
empty_file_roads.GetRasterBand(1).Fill(0)
empty_file_roads.GetRasterBand(1).SetNoDataValue(nodata)
empty_file_roads.FlushCache()
empty_file_roads = None


## **3.4. Rasterizar archivos vectoriales de objeto de interes**

In [ ]:
# se lee un raster de base para obtener parametros generales a utilizar en la rasterizacion de un archivo vectorial
base_raster_vec = f'{DEM_PATH}dem_srtm_500m_32618_final.tif'
bounds = rasterio.open(base_raster_vec ).bounds   
extent = f"{bounds[0]}  {bounds[1]} {bounds[2]} {bounds[3]}"
height, width = rasterio.open(base_raster_vec).shape

In [ ]:
# path a los archivos vectoriales
river_vector_path = f'{PATH_OSM}waterway_epsg32618.gpkg'
roads_vector_path = f'{PATH_OSM}highway_epsge32618.gpkg'

In [ ]:
# funcion para rasterizar los archivos
cmd_rasterize_river = f'gdal_rasterize -burn 255 -ot Byte -ts {width} {height}  -te {extent} {river_vector_path} {PATH_OSM}river_rasterized.tif'
subprocess.run(cmd_rasterize_river, shell=True)

In [ ]:
cmd_rasterize_roads = f'gdal_rasterize -burn 255 -ot Byte -ts {width} {height}  -te {extent} {roads_vector_path} {PATH_OSM}roads_rasterized.tif'
subprocess.run(cmd_rasterize_roads, shell=True)

In [ ]:
# calculo de distancias
values = '255'
maxdist = '50000'
nodata = '-9999'

#distancia a rios
cmd_proximity_rivers = f'gdal_proximity.py -values {values} -distunits GEO -maxdist {maxdist} -nodata {nodata}  {PATH_OSM}river_rasterized.tif {PATH_OSM}distance_from_river.tif -co COMPRESS=DEFLATE -co BIGTIFF=YES -co TILED=YES'
subprocess.run(cmd_proximity_rivers, shell=True)

#distancia a calles
cmd_proximity_roads = f'gdal_proximity.py -values {values} -distunits GEO -maxdist {maxdist} -nodata {nodata}  {PATH_OSM}roads_rasterized.tif {PATH_OSM}distance_from_roads.tif -co COMPRESS=DEFLATE -co BIGTIFF=YES -co TILED=YES'
subprocess.run(cmd_proximity_roads, shell=True)

In [ ]:
#remove files
for f in glob.glob(os.path.join(Path(PATH_OSM, "*_rasterized.tif"))):
    os.remove(f)

--------------------------------------------

# **4. Creacion del dataset**

In [ ]:
DATASET_PATH = f"{BASE_PATH}/dataset/"
!mkdir -p $DATASET_PATH 

In [ ]:
#movemos los archivos generados para el dataset a la carpeta de dataset
#DEM y derivados
shutil.move(f'{DEM_PATH}dem_srtm_500m_32618_final.tif', DATASET_PATH)
shutil.move(f'{DEM_PATH}slope_degrees.tif', DATASET_PATH)
shutil.move(f'{DEM_PATH}twi_topmodel.tif', DATASET_PATH)

#lulc
shutil.move(f'{PATH_LULC_FILES}lulc2021_500m_final.tif', DATASET_PATH)

#distancias
shutil.move(f'{PATH_OSM}distance_from_roads.tif', DATASET_PATH)
shutil.move(f'{PATH_OSM}distance_from_river.tif', DATASET_PATH)

In [ ]:
#mosaico del dataset
files_to_mosaic = glob.glob(os.path.join(DATASET_PATH, "*.tif"))
dataset_files_txt = f'{DATASET_PATH}files.txt'

textfile = open(dataset_files_txt, "w")
for line in files_to_mosaic:
    textfile.write(line + "\n")
textfile.close()

dataset_mosaic_path = f'{DATASET_PATH}dataset.vrt' # ruta de destino del dataset

cmd = f'gdalbuildvrt -separate {dataset_mosaic_path} -input_file_list {dataset_files_txt}'
subprocess.run(cmd, shell=True)

cmd2= f'gdalinfo {dataset_mosaic_path}'
subprocess.run(cmd2, shell=True)

#reproyeccion a EPSG 4326
ds_rp = gdal.Open(f'{DATASET_PATH}dataset.vrt')
output_file = f'{DATASET_PATH}dataset_final.tif'

gdal.UseExceptions()
dsReprj = gdal.Warp(output_file, ds_rp, dstSRS='EPSG:4326')
ds_rp = None
dsReprj = None